<a href="https://colab.research.google.com/github/VictorFrancheto/My_Projects/blob/main/__Construcao_de_modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Aplicando modelo final - fieis 20190830

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import seaborn as sns
pd.options.mode.chained_assignment = None  #hide any pandas warnings

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


### Importa Arquivo

In [ ]:
caminho_arq = 'C:\\Users\\60002991\\_Modelos_dna\\'
df = pd.read_csv(caminho_arq + 'AMOSTRA_FIEL.txt',sep=";", encoding = 'latin-')
df.head()


,CodCli,SAFRA_COMPRA,TP_CARTAO,DtPrimCompra,SEGMENTO,FLG_PF,PF_CANCELADO,flg_saque_facil_ativo,flg_saque_facil_inativo,flg_emprestimo_ativo,...,UF,CodRegiao,CLUSTER,F_CODCA,perc_utilizacao,FX_LIM2,ValLimCred,FLG_CONHECE_EMPRESITMO,fx_perc_utilizacao,TARGET_9
0,2002053536,200010,PL,2000-10-13,02.ALTO,1,0,0,1,0,...,SP,SE,2.0,0,0.000000,a.POLIT_ANTIG,1890.0,1,B - ATE 10%,0
1,2002600309,200010,PL,2000-10-07,02.ALTO,0,0,0,0,0,...,SP,SE,3.0,0,0.306041,a.POLIT_ANTIG,2660.0,0,E - 30% - 40%,0
2,2002600479,200805,BANDEIRA,2018-04-18,03.MEDIO,1,0,1,1,0,...,PR,S,NaN,0,1.052500,a.POLIT_ANTIG,250.0,1,M - > 100%,0
3,2002600924,200010,BANDEIRA,2000-10-25,01.VIP,1,0,0,0,0,...,SP,SE,NaN,0,0.242080,a.POLIT_ANTIG,0.0,0,D - 20% - 30%,1
4,2002600957,200010,PL_COM_BAND_EMITIDO,2000-10-30,03.MEDIO,0,0,0,0,0,...,SP,SE,2.0,0,0.086529,a.POLIT_ANTIG,2400.0,0,B - ATE 10%,1


# Explorando e ajustando base

In [ ]:
#base para criacao do modelo nao precisa ter o id cliente
df = df.copy().set_index('CodCli')

### Ajuste variaveis com missing

In [ ]:
#lista de variaveis com missin
null_values = df.columns[df.isnull().any()]
null_features = df[null_values].isnull().sum().sort_values(ascending = False)
missing_df = pd.DataFrame({'No of Nulls' :null_features})
missing_df

,No of Nulls
CLUSTER,37629
perc_utilizacao,21419


In [ ]:
#transforma missing em -9999
df['CLUSTER'] = df[['CLUSTER']].fillna(-99999)
df['perc_utilizacao'] = df[['perc_utilizacao']].fillna(-99999)

### Agrupamento de variavies categoricas

In [ ]:
#contador de variavel com mais de 4000 observaçoes
df_count_var = df['UF'].fillna('missing').value_counts(dropna= False)
df_count_var[df_count_var>=4000].index

Index(['SP', 'RJ', 'PE', 'BA', 'MG', 'CE', 'RS', 'RN', 'PA', 'PR', 'GO', 'AM',
       'MA', 'DF', 'SC', 'AL', 'PB'],
      dtype='object')

In [ ]:
#agrupa variavel uf e profissao
def create_agrupamento_uf(df):
    if df['UF'] not in [
     'SP', 'RJ', 'PE', 'BA', 'MG', 'RS', 'CE', 'RN', 'PA', 'PR', 'GO', 'AM',
       'MA', 'DF', 'SC', 'AL', 'PB' ]:
        return 'outros'
    else :
        return df['UF']

df_count_var = df['UF'].fillna('missing').value_counts(dropna= False)
df['UF_g'] = df[['UF']].fillna('missing').apply(create_agrupamento_uf,axis= 1)

df[['UF_g','TARGET_9']].fillna('missing').groupby('UF_g').agg({'TARGET_9':[np.mean, np.size]})['TARGET_9'].sort_values(by='mean')



,mean,size
UF_g,,
SC,0.554847,4704
PR,0.594019,8126
RS,0.597595,8648
SP,0.602588,42266
MG,0.602693,11213
RJ,0.604730,19875
DF,0.608569,5858
outros,0.609136,20731
CE,0.610090,8702


In [ ]:
#contador de variavel com mais de 4000 observaçoes
df_count_var = df['DescCategProf'].fillna('missing').value_counts(dropna= False)
df_count_var[df_count_var>=4000].index

Index(['AUTONOMO', 'COMERCIARIOS', 'ASSALARIADOS', 'APOSENTADO E PENSIONISTA',
       'FUNC. PUBLICO MUNICIPAL', 'FUNC. PUBLICO ESTADUAL',
       'APOSENT,PENS,SERV PUBLICO', 'ESTUDANTE', 'PROFESSORES',
       'INDUSTRIARIOS, MESTRES'],
      dtype='object')

In [ ]:
def create_agrupamento_DescCategProf(df):
    if df['DescCategProf'] not in [
       'AUTONOMO', 'COMERCIARIOS', 'ASSALARIADOS', 'APOSENTADO E PENSIONISTA',
       'FUNC. PUBLICO MUNICIPAL', 'FUNC. PUBLICO ESTADUAL',
       'APOSENT,PENS,SERV PUBLICO', 'ESTUDANTE', 'PROFESSORES',
       'INDUSTRIARIOS, MESTRES']:
        return 'outros'
    else :
        return df['DescCategProf']

df_count_var = df['DescCategProf'].fillna('missing').value_counts(dropna= False)
df['DescCategProf_g'] = df[['DescCategProf']].fillna('missing').apply(create_agrupamento_DescCategProf,axis= 1)

df[['DescCategProf_g','TARGET_9']].fillna('missing').groupby('DescCategProf_g').agg({'TARGET_9':[np.mean, np.size]})['TARGET_9'].sort_values(by='mean')


,mean,size
DescCategProf_g,,
"INDUSTRIARIOS, MESTRES",0.568563,4587
ASSALARIADOS,0.594288,27766
COMERCIARIOS,0.596079,31932
AUTONOMO,0.596204,51796
ESTUDANTE,0.613909,5162
APOSENTADO E PENSIONISTA,0.627589,13614
outros,0.629762,36066
FUNC. PUBLICO MUNICIPAL,0.656804,9362
"APOSENT,PENS,SERV PUBLICO",0.665710,5576


In [ ]:
#contador de variavel com mais de 4000 observaçoes
df_count_var = df['DescSitRes'].fillna('missing').value_counts(dropna= False)
df_count_var[df_count_var>=4000].index

Index(['PROPRIA QUITADA', 'COM OS PAIS', 'ALUGADA', 'CEDIDA', 'OUTROS',
       'PROPRIA FINANCIADA'],
      dtype='object')

In [ ]:
def create_agrupamento_DescSitRes(df):
    if df['DescSitRes'] not in [
       'PROPRIA QUITADA', 'COM OS PAIS', 'ALUGADA', 'CEDIDA', 'OUTROS',
       'PROPRIA FINANCIADA' ]:
        return 'outros'
    else :
        return df['DescSitRes']

df_count_var = df['DescSitRes'].fillna('missing').value_counts(dropna= False)
df['DescSitRes_g'] = df[['DescSitRes']].fillna('missing').apply(create_agrupamento_DescSitRes,axis= 1)
df[['DescSitRes_g','TARGET_9']].fillna('missing').groupby('DescSitRes_g').agg({'TARGET_9':[np.mean, np.size]})['TARGET_9'].sort_values(by='mean')


,mean,size
DescSitRes_g,,
CEDIDA,0.567160,11078
OUTROS,0.579344,10486
ALUGADA,0.591743,22623
COM OS PAIS,0.595922,34033
outros,0.601266,158
PROPRIA QUITADA,0.630513,116383
PROPRIA FINANCIADA,0.633709,5239


In [ ]:
#contador de variavel com mais de 4000 observaçoes
df_count_var = df['DescEscol'].fillna('missing').value_counts(dropna= False)
df_count_var[df_count_var>=4000].index

Index(['SEGUNDO GRAU COMPLETO', 'SUPERIOR COMPLETO', 'SUPERIOR INCOMPLETO',
       'PRIMEIRO GRAU COMPLETO', 'SEGUNDO GRAU INCOMPLETO',
       'PRIMEIRO GRAU INCOMPLETO', 'POS-GRADUACAO COMPLETA'],
      dtype='object')

In [ ]:
def create_agrupamento_DescEscol(df):
    if df['DescEscol'] not in [
        'SEGUNDO GRAU COMPLETO', 'SUPERIOR COMPLETO', 'SUPERIOR INCOMPLETO',
        'PRIMEIRO GRAU COMPLETO', 'SEGUNDO GRAU INCOMPLETO',
        'PRIMEIRO GRAU INCOMPLETO', 'POS-GRADUACAO COMPLETA' ]:
        return 'outros'
    else :
        return df['DescEscol']

df_count_var = df['DescEscol'].fillna('missing').value_counts(dropna= False)
df['DescEscol_g'] = df[['DescEscol']].fillna('missing').apply(create_agrupamento_DescEscol,axis= 1)
df[['DescEscol_g','TARGET_9']].fillna('missing').groupby('DescEscol_g').agg({'TARGET_9':[np.mean, np.size]})['TARGET_9'].sort_values(by='mean')

,mean,size
DescEscol_g,,
SEGUNDO GRAU INCOMPLETO,0.579703,9617
PRIMEIRO GRAU INCOMPLETO,0.590462,8031
PRIMEIRO GRAU COMPLETO,0.595113,11418
SEGUNDO GRAU COMPLETO,0.605289,100856
SUPERIOR INCOMPLETO,0.613762,24881
outros,0.627928,1110
SUPERIOR COMPLETO,0.649136,38713
POS-GRADUACAO COMPLETA,0.663379,5374


In [ ]:
#apaga variaveis
df = df.drop(['DescCategProf','UF','DtPrimCompra','DescSitRes','DescEscol'], axis=1)


### Transforma variaveis numericas em categoricas

In [ ]:
##RECALSSIFICA AS VARIAVEIS

df['CLUSTER'][df['CLUSTER'] == -99999] = 'S/CLUSTER'
df['CLUSTER'][df['CLUSTER'] == 1] = 'CLUSTER_1'
df['CLUSTER'][df['CLUSTER'] == 2] = 'CLUSTER_2'
df['CLUSTER'][df['CLUSTER'] == 3] = 'CLUSTER_3'
df['CLUSTER'][df['CLUSTER'] == 4] = 'CLUSTER_4'
df['CLUSTER'][df['CLUSTER'] == 5] = 'CLUSTER_5'
df['CLUSTER'][df['CLUSTER'] == 6] = 'CLUSTER_6'
df['CLUSTER'][df['CLUSTER'] == 7] = 'CLUSTER_7'

df['IndVeiculo'][df['IndVeiculo'] == 0] = 'S/VEICULO'
df['IndVeiculo'][df['IndVeiculo'] == 1] = 'C/VEICULO'

df['F_CODCA'][df['F_CODCA'] == 0] = 'S/ATRASO'
df['F_CODCA'][df['F_CODCA'] == 1] = 'C/ATRASO'
df['F_CODCA'][df['F_CODCA'] == -99999] = 'S/INF_ATRASO'

df['FLAG_EMAIL'][df['FLAG_EMAIL'] == 0] = 'S/EMAIL'
df['FLAG_EMAIL'][df['FLAG_EMAIL'] == 1] = 'C/EMAIL'

df['FLG_CONHECE_EMPRESITMO'][df['FLG_CONHECE_EMPRESITMO'] == 0] = 'NAO CONHECE EMPRESTIMO'
df['FLG_CONHECE_EMPRESITMO'][df['FLG_CONHECE_EMPRESITMO'] == 1] = 'CONHECE EMPRESTIMO'

df['FLG_PF'][df['FLG_PF'] == 0] = 'S/PF'
df['FLG_PF'][df['FLG_PF'] == 1] = 'C/PF'

df['PF_CANCELADO'][df['PF_CANCELADO'] == 0] = 'S/PF_CANCELADO'
df['PF_CANCELADO'][df['PF_CANCELADO'] == 1] = 'C/PF_CANCELADO'

df['flg_emprestimo_ativo'][df['flg_emprestimo_ativo'] == 0] = 'S/EMP_ATIVO'
df['flg_emprestimo_ativo'][df['flg_emprestimo_ativo'] == 1] = 'C/EMP_ATIVO'

df['flg_emprestimo_inativo'][df['flg_emprestimo_inativo'] == 0] = 'S/EMP_INATIVO'
df['flg_emprestimo_inativo'][df['flg_emprestimo_inativo'] == 1] = 'C/EMP_INATIVO'

df['flg_saque_facil_ativo'][df['flg_saque_facil_ativo'] == 0] = 'S/SAQUE_FACIL_ATIVO'
df['flg_saque_facil_ativo'][df['flg_saque_facil_ativo'] == 1] = 'C/SAQUE_FACIL_ATIVO'

df['flg_saque_facil_inativo'][df['flg_saque_facil_inativo'] == 0] = 'S/SAQUE_FACIL_INATIVO'
df['flg_saque_facil_inativo'][df['flg_saque_facil_inativo'] == 1] = 'C/SAQUE_FACIL_INATIVO'



### Separando as variáveis

In [ ]:
features_numeric =[]
features_char = []

for var in df.columns:
    if (var != 'TARGET_9') :
        if df.dtypes[var] == 'O':
            features_char.append(var)
        else:
            features_numeric.append(var)
print( ' Variáveis categóricas : ')
print(features_char)
print( ' Variáveis numéricas : ')
print(features_numeric)

 Variáveis categóricas : 
['TP_CARTAO', 'SEGMENTO', 'FLG_PF', 'PF_CANCELADO', 'flg_saque_facil_ativo', 'flg_saque_facil_inativo', 'flg_emprestimo_ativo', 'flg_emprestimo_inativo', 'IndVeiculo', 'FLAG_EMAIL', 'DescTipRes', 'DescEstCivil', 'DescSex', 'CodRegiao', 'CLUSTER', 'F_CODCA', 'FX_LIM2', 'FLG_CONHECE_EMPRESITMO', 'fx_perc_utilizacao', 'UF_g', 'DescCategProf_g', 'DescSitRes_g', 'DescEscol_g']
 Variáveis numéricas : 
['SAFRA_COMPRA', 'QtdFilho', 'QtdFilhoMenor', 'ValRendaTit', 'ValRendaFam', 'IndCojg', 'IDADE', 'ANOS_CADASTRO', 'IndPop', 'perc_utilizacao', 'ValLimCred']


In [ ]:
def analise_cat_1(df,var,label = '',fl_ordena=0, num = False, q = 0, q2 = 0, y = 'TARGET_9'):
    if label == '':
        label = var
    df_ = df.copy()
    if num:
        if q == 0:
            if q2 == 0:
                df_[var+'_cat'] = pd.qcut(df_[var],q= [0.0,0.2,0.5,0.8,1.0],retbins=True,duplicates='drop')[0].cat.add_categories('missing')
                var= var+'_cat'
            else:
                df_[var+'_cat'] = pd.qcut(df_[var],q= q2,retbins=True,duplicates='drop')[0].cat.add_categories('missing')
                var= var+'_cat'
        else :
            df_[var+'_cat'] = pd.cut(df_[var],bins=q).cat.add_categories('missing')
            var= var+'_cat'
    print('\n')
    print('Análise da variável {}'.format(var))
    print('# valores distintos {}'.format(df_[var].nunique()))
    try:
        df_[var] = df_[var].fillna('missing')
    except :
        df_[var] = df_[var].cat.add_categories('missing').fillna('missing')

    total_compra = df_[y].sum()
    total_sem_compra = df_.shape[0] - df_[y].sum()

    def tx(x):
        return sum(x)*100/sum(1 -x +x)
    def pct(x):
        return sum(1-x+x)*1.0/ (df_.shape[0])
    def pct_compra(x):
        return (sum(x)/total_compra )
    def rr(x):
        return (sum(x)/total_compra )/(sum(1-x)/total_sem_compra)
    def WoE(x):
        return np.log(rr(x))
    def IV(x):
        return -(sum(1-x)/total_sem_compra - sum(x)/total_compra)*WoE(x)
    s = df_.groupby(var).agg({y: [np.size,pct,pct_maus, tx,rr,WoE,IV]})[y]
    def color_negative_green(val):
        color = 'green' if val < 0 else 'black'
        return 'color: %s' % color
    print('IV : {:.3f}'.format(s.IV.sum()))
    #print('Beta : {:.3f}'.format(dic_beta[var]))
    t = s.style.applymap(color_negative_green)
    display(pd.DataFrame(df_[var].value_counts()).join(pd.DataFrame(df_[var].value_counts(normalize= True)*100), lsuffix = 'k').rename(columns = {var+'k': '#', var: '%'}))
    display(t)
    by_var = df_.groupby(var).agg({y:[np.size, np.mean]})[y]
    if fl_ordena ==1 :
        by_var =  by_var.sort_values(by = 'mean')
    Y1 = by_var['size']
    Y2 =by_var['mean']
    Y_mean = np.ones(shape=(len(Y1.index)))* df_[y].mean()
    index =  np.arange(len(Y1.index))
    with plt.style.context('my_custom_style'):
        plt.bar(index,Y1,alpha = 0.3, color= 'gray')
        plt.grid(False)
        plt.xticks(rotation = 20         if var[:2] not in ( 'fl', 'cd')  else 0)
        plt.ylabel('# registros')
        plt.twinx()
        plt.gca().set_xticks(index)
        plt.gca().set_xticklabels([Y1.index[i] for i in index], rotation = 40)
        plt.plot(index,Y_mean,label=  'tx. média evento')
        plt.plot(index,Y2,marker = 'o',label=  'tx. evento')
        plt.gca().set_yticklabels([ ' {:.2f}%'.format( i*100) for i in plt.gca().get_yticks()])
        plt.grid(False)
        plt.title('Bivariada  {}'.format(label))
        plt.ylabel('tx. evento')
        plt.xlabel(label)
        if var[:2] in ( 'fl', 'cd'):
            plt.legend(loc= 9,bbox_to_anchor=(0.5, -0.1))
        else:
            plt.legend(loc= 9,bbox_to_anchor=(0.5, -0.35))
        plt.show()
        print(s.IV.sum())
        return  (var , s.IV.sum())
        print('\n')

In [ ]:
total_bons = df['TARGET_9'].sum()
total_maus = df.shape[0] - total_bons

print('Total de bons : {} \nTotal de maus : {}'.format(total_bons,total_maus))

Total de bons : 122822 
Total de maus : 77178


In [ ]:

for var in features_char:
    df[var+'_woe'] = np.nan
    s=df[[var, 'TARGET_9']].fillna('missing').groupby(var).agg({'TARGET_9':[np.mean, np.size, lambda x : np.log((sum(1-x)/total_bons)/(sum(x)/total_maus))   ]}).rename(columns={'mean': 'tx_evento', 'size': 'N', '<lambda>': 'woe'})['TARGET_9'].sort_values(by='woe')
    display(s)
    for i in s.index:
        df.loc[df[var] == i,[var+'_woe'] ]= s.loc[i]['woe']

NameError: name 'TP_CARTAO' is not defined

## Transforma variaveis categoricas em dummy

In [ ]:
pd.options.mode.chained_assignment = None  #hide any pandas warnings
df = pd.get_dummies(df, drop_first=True)

In [ ]:
from sklearn.model_selection  import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop('TARGET_9', 1), df['TARGET_9'], test_size = .2, random_state=10)
#split the data

In [ ]:
def calcula_indicadores(est, nome_est):

    y_pred_ = [i[1] for i in est.predict_proba(X_test)]
    y_test_ = y_test
    t = pd.DataFrame( {'fl_resposta':y_test_, 'p': y_pred_})

    ##### Cálculo do KS #######

    ks, std_ks = stats.ks_2samp(t.loc[t.fl_resposta == 0,:]['p'], t.loc[t.fl_resposta == 1,:]['p'])

    fpr, tpr, _ = roc_curve(y_test_, y_pred_)
    try:
        with plt.style.context('my_custom_style'):
            plt.plot([0, 1], [0, 1], 'k--')
            plt.plot(fpr, tpr, label=nome_est)
            plt.xlabel('False positive rate')
            plt.ylabel('True positive rate')
            plt.title('ROC curve')
            plt.legend(loc='best')
    except:
        plt.plot([0, 1], [0, 1], 'k--')
        plt.plot(fpr, tpr, label=nome_est)
        plt.xlabel('False positive rate')
        plt.ylabel('True positive rate')
        plt.title('ROC curve')
        plt.legend(loc='best')

    return [nome_est, round((2*roc_auc_score(y_test_,y_pred_) -1)*100,2) ,round(ks*100,2) ,round(log_loss(y_test_, y_pred_,normalize= False),2) ]

In [ ]:
from sklearn.metrics import roc_curve, log_loss,roc_auc_score
from scipy import stats
from IPython.display import display

def avalia_modelo(estimador):
###### público da performance #######
    lista_resultados=[]
    for est in estimador.keys():
        lista_resultados.append(calcula_indicadores(estimador[est], est))
    display(pd.DataFrame(lista_resultados, columns= ['Modelo', 'Gini', 'KS', 'logloss']))
    plt.show()

In [ ]:
# 1. XGBoost
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.4, gamma=0.05, learning_rate=0.15,
       max_delta_step=0, max_depth=7, min_child_weight=10, missing=-1,
       n_estimators=300, n_jobs=16,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
       subsample=1.0)

xgb.fit(X_train,y_train)
avalia_modelo({'xgb':xgb})


In [ ]:
xgb_clf = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.4, gamma=0.01, learning_rate=0.075,
       max_delta_step=0, max_depth=5, min_child_weight=10, missing=-1,
       n_estimators=300, n_jobs=16,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
       subsample=0.8)

xgb_clf.fit(X_train,y_train)

avalia_modelo({'xgb':xgb, 'xgb_clf':xgb_clf})


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf= RandomForestClassifier(bootstrap=False, class_weight= None, criterion='gini',
            max_depth=14, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=250,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=-1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

rf.fit(X_train,y_train)


In [ ]:
avalia_modelo({'xgb':xgb, 'xgb_clf':xgb_clf, 'rf ':rf })

In [ ]:
importances = xgb.feature_importances_
indices = np.argsort(importances)[::-1]
h = X_train.columns
print("Feature ranking:")
lables=[]
for f in range((X_train.shape[1])):
    l= h[indices[f]]
    lables.append(l)
    print("%d. feature %s (%f)" % (f + 1,l, importances[indices[f]]))

In [ ]:
#Prevê o resultado
classifier = xgb.fit(X_train,y_train)
predictions =  classifier.predict_proba(X_test)[:,1]
print(predictions)


In [ ]:
df.shape

# escorando a base

In [ ]:
#importa base de validacao

df_tratado_test = pd.read_csv(caminho_arq+'VALIDA_FIEIS_TARGET_9.txt', sep=";", encoding = 'latin-')

In [ ]:
df_tratado_test = df_tratado_test.copy().set_index('CodCli')
#print(df_tratado_test
#df_tratado_test.head(10)
df_tratado_test.shape

In [ ]:
#apaga variaveis -  base para criacao do modelo nao precisa ter o id cliente


#transforma missing em -9999
df_tratado_test['CLUSTER'] = df_tratado_test[['CLUSTER']].fillna(-99999)
df_tratado_test['F_CODCA'] = df_tratado_test[['F_CODCA']].fillna(-99999)
df_tratado_test['perc_utilizacao'] = df_tratado_test[['perc_utilizacao']].fillna(-99999)

#agrupa variavel uf e profissao
def create_agrupamento_uf(df_tratado_test):
    if df_tratado_test['UF'] not in [
     'SP', 'RJ', 'PE', 'BA', 'MG', 'RS', 'CE', 'RN', 'PA', 'PR', 'GO', 'AM',
       'MA', 'DF', 'SC', 'AL', 'PB' ]:
        return 'outros'
    else :
        return df_tratado_test['UF']

df_tratado_test_count_var = df_tratado_test['UF'].fillna('missing').value_counts(dropna= False)
df_tratado_test['UF_g'] = df_tratado_test[['UF']].fillna('missing').apply(create_agrupamento_uf,axis= 1)

def create_agrupamento_DescCategProf(df_tratado_test):
    if df_tratado_test['DescCategProf'] not in [
    'AUTONOMO', 'COMERCIARIOS', 'ASSALARIADOS', 'APOSENTADO E PENSIONISTA',
       'FUNC. PUBLICO MUNICIPAL', 'FUNC. PUBLICO ESTADUAL',
       'APOSENT,PENS,SERV PUBLICO', 'ESTUDANTE', 'PROFESSORES',
       'INDUSTRIARIOS, MESTRES', 'FUNCIONARIO PUBLICO', 'EMPRESARIO',
       'PREST DE SERVICOS - SAUDE', 'DO LAR', 'EMPREGADOS DOMESTICOS',
       'PROFISSIONAL LIBERAL']:
        return 'outros'
    else :
        return df_tratado_test['DescCategProf']

df_tratado_test_count_var = df_tratado_test['DescCategProf'].fillna('missing').value_counts(dropna= False)
df_tratado_test['DescCategProf_g'] = df_tratado_test[['DescCategProf']].fillna('missing').apply(create_agrupamento_DescCategProf,axis= 1)


def create_agrupamento_DescSitRes(df_tratado_test):
    if df_tratado_test['DescSitRes'] not in [
       'PROPRIA QUITADA', 'COM OS PAIS', 'ALUGADA', 'CEDIDA', 'OUTROS',
       'PROPRIA FINANCIADA' ]:
        return 'outros'
    else :
        return df_tratado_test['DescSitRes']

df_tratado_test_count_var = df_tratado_test['DescSitRes'].fillna('missing').value_counts(dropna= False)
df_tratado_test['DescSitRes_g'] = df_tratado_test[['DescSitRes']].fillna('missing').apply(create_agrupamento_DescSitRes,axis= 1)


def create_agrupamento_DescEscol(df_tratado_test):
    if df_tratado_test['DescEscol'] not in [
        'SEGUNDO GRAU COMPLETO', 'SUPERIOR COMPLETO', 'SUPERIOR INCOMPLETO',
        'PRIMEIRO GRAU COMPLETO', 'SEGUNDO GRAU INCOMPLETO',
        'PRIMEIRO GRAU INCOMPLETO', 'POS-GRADUACAO COMPLETA' ]:
        return 'outros'
    else :
        return df_tratado_test['DescEscol']

df_tratado_test_count_var = df_tratado_test['DescEscol'].fillna('missing').value_counts(dropna= False)
df_tratado_test['DescEscol_g'] = df_tratado_test[['DescEscol']].fillna('missing').apply(create_agrupamento_DescEscol,axis= 1)

#apaga variaveis
df_tratado_test = df_tratado_test.drop(['DescCategProf','UF','DtPrimCompra','DescEscol','DescSitRes'], axis=1)


##RECALSSIFICA AS VARIAVEIS

df_tratado_test['CLUSTER'][df_tratado_test['CLUSTER'] == -99999] = 'S/CLUSTER'
df_tratado_test['CLUSTER'][df_tratado_test['CLUSTER'] == 1] = 'CLUSTER_1'
df_tratado_test['CLUSTER'][df_tratado_test['CLUSTER'] == 2] = 'CLUSTER_2'
df_tratado_test['CLUSTER'][df_tratado_test['CLUSTER'] == 3] = 'CLUSTER_3'
df_tratado_test['CLUSTER'][df_tratado_test['CLUSTER'] == 4] = 'CLUSTER_4'
df_tratado_test['CLUSTER'][df_tratado_test['CLUSTER'] == 5] = 'CLUSTER_5'
df_tratado_test['CLUSTER'][df_tratado_test['CLUSTER'] == 6] = 'CLUSTER_6'
df_tratado_test['CLUSTER'][df_tratado_test['CLUSTER'] == 7] = 'CLUSTER_7'

df_tratado_test['IndVeiculo'][df_tratado_test['IndVeiculo'] == 0] = 'S/VEICULO'
df_tratado_test['IndVeiculo'][df_tratado_test['IndVeiculo'] == 1] = 'C/VEICULO'

df_tratado_test['F_CODCA'][df_tratado_test['F_CODCA'] == 0] = 'S/ATRASO'
df_tratado_test['F_CODCA'][df_tratado_test['F_CODCA'] == 1] = 'C/ATRASO'
df_tratado_test['F_CODCA'][df_tratado_test['F_CODCA'] == -99999] = 'S/INF_ATRASO'

df_tratado_test['FLAG_EMAIL'][df_tratado_test['FLAG_EMAIL'] == 0] = 'S/EMAIL'
df_tratado_test['FLAG_EMAIL'][df_tratado_test['FLAG_EMAIL'] == 1] = 'C/EMAIL'

df_tratado_test['FLG_CONHECE_EMPRESITMO'][df_tratado_test['FLG_CONHECE_EMPRESITMO'] == 0] = 'NAO CONHECE EMPRESTIMO'
df_tratado_test['FLG_CONHECE_EMPRESITMO'][df_tratado_test['FLG_CONHECE_EMPRESITMO'] == 1] = 'CONHECE EMPRESTIMO'

df_tratado_test['FLG_PF'][df_tratado_test['FLG_PF'] == 0] = 'S/PF'
df_tratado_test['FLG_PF'][df_tratado_test['FLG_PF'] == 1] = 'C/PF'

df_tratado_test['PF_CANCELADO'][df_tratado_test['PF_CANCELADO'] == 0] = 'S/PF_CANCELADO'
df_tratado_test['PF_CANCELADO'][df_tratado_test['PF_CANCELADO'] == 1] = 'C/PF_CANCELADO'

df_tratado_test['flg_emprestimo_ativo'][df_tratado_test['flg_emprestimo_ativo'] == 0] = 'S/EMP_ATIVO'
df_tratado_test['flg_emprestimo_ativo'][df_tratado_test['flg_emprestimo_ativo'] == 1] = 'C/EMP_ATIVO'

df_tratado_test['flg_emprestimo_inativo'][df_tratado_test['flg_emprestimo_inativo'] == 0] = 'S/EMP_INATIVO'
df_tratado_test['flg_emprestimo_inativo'][df_tratado_test['flg_emprestimo_inativo'] == 1] = 'C/EMP_INATIVO'

df_tratado_test['flg_saque_facil_ativo'][df_tratado_test['flg_saque_facil_ativo'] == 0] = 'S/SAQUE_FACIL_ATIVO'
df_tratado_test['flg_saque_facil_ativo'][df_tratado_test['flg_saque_facil_ativo'] == 1] = 'C/SAQUE_FACIL_ATIVO'

df_tratado_test['flg_saque_facil_inativo'][df_tratado_test['flg_saque_facil_inativo'] == 0] = 'S/SAQUE_FACIL_INATIVO'
df_tratado_test['flg_saque_facil_inativo'][df_tratado_test['flg_saque_facil_inativo'] == 1] = 'C/SAQUE_FACIL_INATIVO'

# Transformando as variaveis catergoricas em dummy
pd.options.mode.chained_assignment = None  #hide any pandas warnings
df_tratado_test = pd.get_dummies(df_tratado_test, drop_first=True)



In [ ]:
df_tratado_test.shape

In [ ]:
#apaga variaveis
df_tratado_test = df_tratado_test.drop(['DescTipRes_NÃO IDENTIFICADO'], axis=1)



## Checando as variáveis

In [ ]:
for var in df_tratado_test.columns:
    if var not in df.columns:
        print(var)


In [ ]:
for var in df.columns:
    if var not in df_tratado_test.columns:
        print(var)

In [ ]:
print('The size of the training set: ', df.shape)
print('The size of the training set: ', df_tratado_test.shape)

In [ ]:
predictions2 =  classifier.predict_proba(df_tratado_test)[:,1]
print(predictions2)

In [ ]:
fieis_submission = pd.DataFrame ({"CodCli": df_tratado_test.index,
                                  "Target": predictions2})


In [ ]:
fieis_submission.to_csv('C:\\Users\\60002991\\_Modelos_dna\\piroca.txt', index = False)

fieis_submission.sample(10)